In [1]:
import pandas as pd
import numpy as np

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
bought_list = [521, 32, 143, 991]
prices_recommended = [400, 60, 40, 40 , 90]

**Hit rate@k**

In [3]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    """
    Parameters
    ----------
    recommended_list: list
        Список с рекомендациями для пользователя
    bought_list: list
        Список покупок пользователя
    k: int
        Количество рекомендаций
        
    Returns
    -------
    hit_rate: int 
        Количество релевантных товаров из топ-k рекомендованных
    
    """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list 
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = flags.sum() 
    
    return hit_rate

In [4]:
hit_rate_at_k(recommended_list, bought_list, 5)

2

**Money Precision@k**

In [5]:
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    """
    Parameters
    ----------
    recommended_list: list
        Список с рекомендациями для пользователя
    bought_list: list
        Список покупок пользователя
    prices_recommended: list
        Список рекомендуемых цен
    k: int
        Количество рекомендаций
        
    Returns
    -------
    precision: float 
        Доход от релевантных товаров среди топ-k рекомендованных
    
    """
        
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    precision = flags*1@prices_recommended[:k] / np.sum(prices_recommended[:k])
    
    return precision

In [6]:
money_precision_at_k(recommended_list, bought_list, prices_recommended, k=4)

0.14814814814814814

**Recall@k, Money Recall@k**

In [7]:
def recall_at_k(recommended_list, bought_list, k=5):
    """
    Parameters
    ----------
    recommended_list: list
        Список с рекомендациями для пользователя
    bought_list: list
        Список покупок пользователя
    k: int
        Количество рекомендаций
        
    Returns
    -------
    recall: float
        Количество релевантных рекомендаций среди топ-k рекомендованных
    
    """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    """
    Parameters
    ----------
    recommended_list: list
        Список с рекомендациями для пользователя
    bought_list: list
        Список покупок пользователя
    prices_recommended: list
        Список рекомендуемых цен
    prices_bought: list
        Список цен в чеке клиента
    k: int
        Количество рекомендаций
        
    Returns
    -------
    recall: float 
        Доход от релевантных рекомендаций среди топ-k рекомендованных
    """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    recall = flags*1@prices_recommended[:k] / (bought_list@prices_bought)
    
    return recall

**MAP@k¶**

In [9]:
def ap_k(recommended_list, bought_list, k=5):
    
    """
    Parameters
    ----------
    recommended_list: list
        список с рекомендациями для пользователя
    bought_list: list
        список покупок пользователя
    k: int
        количество наблюдений
        
    Returns
    -------
    result: float 
        Среднее суммы точностей по К индексам для только релевантных элементов
    
    """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [14]:
def map_k(recommended_list, bought_list, k=5, u=100):
    
    """
    Parameters
    ----------
    recommended_list: list
        список списков с рекомендациями для всех пользователей
    bought_list: list
        список списков с покупками всех пользователей
    k: int
        количество наблюдений
    u: int
        количество пользователей
        
    Returns
    -------
    result: float
        Среднее AP@k по всем пользователям    
    
    """
    
    sum_ = 0
    for user in range(u):
        user_ap_k = ap_k(recommended_list[user], bought_list[user], k)
        sum_+= user_ap_k
    
    result = sum_/u
    
    return result

**RR**

In [15]:
def reciprocal_rank(recommended_list, bought_list):
    
    """
    Parameters
    ----------
    recommended_list: list
        список списков с рекомендациями для всех пользователей
    bought_list: list
        список списков с покупками всех пользователей
   
    Returns
    -------
    result: float
        Обратный ранг первого правильно рекомендованного элемента    
    
    """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
       
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    ranks, = np.where(flags == True)
    
    result = 1 / (ranks[0]+1)
    
    return result

In [16]:
recommended_list = [156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [17]:
reciprocal_rank(recommended_list, bought_list)

0.3333333333333333

**MRR@k**

In [18]:
def mean_reciprocal_rank(recommended_list, bought_list, k=5):
    
    """
    Parameters
    ----------
    recommended_list: list
        список списков с рекомендациями для всех пользователей
    bought_list: list
        список списков с покупками всех пользователей
    k: int
        количество наблюдений
            
    Returns
    -------
    result: float
        Среднее обратных рангов всех правильно рекомендованных элементов     
    
    """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
       
    flags = np.isin(recommended_list[:k], bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(k):
        
        if flags[i]:
            rr = 1/(i+1)
            sum_ += rr
            
    result = sum_ / k
    
    return result

In [19]:
recommended_list = [156, 143, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [20]:
mean_reciprocal_rank(recommended_list, bought_list, k=5)

0.15